# Initial Tests of Assumed Shapes Model

For the first tests will test the model against the case in which there is no FWTD

In [1]:
from FwtModels import AssumedShape
import numpy as np
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import y,x

## Create the Symbolic Model & show it is equivelent to that in the book

Within Jonathans book (section 11.6) the following equation is 'derived' for a rectangluar wing with one mode in both bending and torsion

$m\left[\begin{array}{cc}{\frac{c s^{5}}{5}} & {\frac{s^{4}}{4}\left(\frac{c^{2}}{2}-c x_{\mathrm{f}}\right)} \\ {\frac{s^{4}}{4}\left(\frac{c^{2}}{2}-c x_{\mathrm{f}}\right)} & {\frac{s^{3}}{3}\left(\frac{c^{3}}{3}-c^{2} x_{\mathrm{f}}+c x_{\mathrm{f}}^{2}\right)}\end{array}\right]\left\{\begin{array}{c}{\dot{q}_{1}} \\ {\ddot{q}_{2}}\end{array}\right\}+\rho V\left[\begin{array}{cc}{\frac{c a_{\mathrm{WS}} 5}{10}} & {0} \\ {-\frac{c^{2} e a_{\mathrm{WS}}^{4}}{8}} & {-\frac{c^{3} s^{3} M_{\theta}}{24}}\end{array}\right]\left\{\begin{array}{c}{\dot{q}_{1}} \\ {\dot{q}_{2}}\end{array}\right\}+\left(\rho V^{2}\left[\begin{array}{cc}{0} & {\frac{c s^{4} a_{\mathrm{W}}}{8}} \\ {0} & {-\frac{e c^{2} s^{3} a_{W}}{6}}\end{array}\right]+\left[\begin{array}{cc}{4 E I s} & {0} \\ {0} & {G J s}\end{array}\right]\right)\left\{\begin{array}{l}{q_{1}} \\ {q_{2}}\end{array}\right\}=\left\{\begin{array}{l}{0} \\ {0}\end{array}\right\}$

This section will aim to recreat each of these equations from my generalised model by setting the FWTD's size to 0

First, need to generate a symbolic model for the generalised case

In [8]:
# create generalised coordinates
q0,q1,q2 = me.dynamicsymbols('q:3')
q = sym.Matrix([q0,q1,q2])

# create instance of Fwt parameters
p = AssumedShape.FwtParameters()

# define shape Functions
z_w = q0*y**2 + q1*(x-p.x_f)*y
alpha_w = q1 * y
z_t = z_w.subs(y,p.s_w) + q2*(y)
alpha_t = alpha_w.subs(y,p.s_w) + q2*sym.sin(p.Lambda)

# create the model
sModel = AssumedShape.SymbolicModel(q,z_w,alpha_w,z_t,alpha_t,p)
sModel.GenerateEoM()

for each matrix set $s_t$ to zero and print the result to compare with the original equation

In [9]:
# Damping Matrix, divide by M_t as this is outside the matrix above
(-sModel.B.subs(p.s_t,0))/(p.rho*p.V)

⎡           5                     ⎤
⎢  a_w⋅c⋅s_w                      ⎥
⎢  ──────────           0        0⎥
⎢      10                         ⎥
⎢                                 ⎥
⎢      2      4         3    3    ⎥
⎢-a_w⋅c ⋅e⋅s_w    -M_α̇⋅c ⋅s_w     ⎥
⎢───────────────  ─────────────  0⎥
⎢       8               24        ⎥
⎢                                 ⎥
⎣       0               0        0⎦

In [10]:
# Mass Matrix, divide by rho*V as this is outside the matrix above
sym.simplify((sModel.M/p.m_w).subs(p.s_t,0))

⎡           5                   4                     ⎤
⎢      c⋅s_w               c⋅s_w ⋅(c - 2⋅x_f)         ⎥
⎢      ──────              ──────────────────        0⎥
⎢        5                         8                  ⎥
⎢                                                     ⎥
⎢     4                   3 ⎛ 2                  2⎞   ⎥
⎢c⋅s_w ⋅(c - 2⋅x_f)  c⋅s_w ⋅⎝c  - 3⋅c⋅x_f + 3⋅x_f ⎠   ⎥
⎢──────────────────  ──────────────────────────────  0⎥
⎢        8                         9                  ⎥
⎢                                                     ⎥
⎣        0                         0                 0⎦

In [11]:
# Aero Stiffness Matrix, divide by rho*V^2 as this is outside the matrix above
(-sModel.C.subs(p.s_t,0))/(p.rho*p.V**2)

⎡              4      ⎤
⎢     a_w⋅c⋅s_w       ⎥
⎢0    ──────────     0⎥
⎢         8           ⎥
⎢                     ⎥
⎢         2      3    ⎥
⎢   -a_w⋅c ⋅e⋅s_w     ⎥
⎢0  ───────────────  0⎥
⎢          6          ⎥
⎢                     ⎥
⎣0         0         0⎦

In [12]:
# Structural Stiffness Matrix
(sModel.K.subs({p.s_t:0, p.k_theta:0}))

⎡4⋅EI⋅s_w    0     0⎤
⎢                   ⎥
⎢   0      GJ⋅s_w  0⎥
⎢                   ⎥
⎣   0        0     0⎦

Inspection shows all of these matrices are the same as that from the book with the exception of the aero samping matrix (bottum left reads a_w^4*s in the book which is a mistake as no mechanism in the maths coan really raise a_w to the power of anything)

## Test a couple of Numeric Models

### Compare Frequency values for a SDoF system

with Jonathan's book, in section 3.2.1, a model is built of a continous wing with a single assumed shape:

$z(y, t)=\psi(y) q(t)=\left(\frac{y}{s}\right)^{2} q(t)$

The natural frequency of this system is found to be $\omega=4.47 \sqrt{\frac{E I}{\mu s^{4}}}$ and the exact answer is shown as $\omega=3.516 \sqrt{\frac{E I}{\mu s^{4}}}$

#### Compare model with the answer for one assumed shape in bending

In [ ]:
# create generalised coordinates
q0,q1 = me.dynamicsymbols('q:2') # Model re
q = sym.Matrix([q0,q1])

# create instance of Fwt parameters
p = AssumedShape.FwtParameters()

# define shape Functions
z_w = q0*y**2
alpha_w = sym.Rational(0,1)
z_t = z_w.subs(y,p.s_w) + q1*(y)
alpha_t = alpha_w.subs(y,p.s_w) + q1*sym.sin(p.Lambda)

# create the model
sModel = AssumedShape.SymbolicModel(q,z_w,alpha_w,z_t,alpha_t,p)
sModel.GenerateEoM()

### Use to estimate frequencies of the WTT's

Will use parameter from the wing used in the WTT's in December, it had a span of 1.345m and the structural properties close match those of the steel beam at its quarter-cord (5mm x 30mm cross section). Assume a uniform mass distrobution and a total weight of 2.75Kg. (the FWT took up 29% of the wetted area, but for now we are modelling as a continous beam)

In [7]:
semiSpan = 1.345  # semi-span of the wing
ratioFwt = 0   # ratio of the wing semi span that is fwt
wingMass = 2.75   # mass of entire semi-span wing
chord = 0.15

mu = wingMass/(chord*semiSpan) # mass density of the wing

In [8]:
# modeling the structural properties off a rectangular bar

# with width b and hieght h
h = 0.005 # hieght of bar
b = 0.03 # width of bar
I_xx = b*h**3/12
I_yy = b**3*h/12
J = I_xx + I_yy

# Material properties
E = 190e9
G = 74e9

In [9]:
# create an instance of the class holding all the properties for the FWT
p = aShape.FwtParameters(m_w = mu,
                        m_t = mu,
                        x_f = 0.25,
                        s_w = semiSpan*(1-ratioFwt),
                        s_t = semiSpan*ratioFwt,
                        c = chord,
                        Lambda = np.deg2rad(10),
                        EI = E*I_xx,
                        GJ = G*J,
                        k_theta = 0,
                        rho = 1.225,
                        V = 10,
                        a_w = 2*np.pi,
                        a_t = 2*np.pi,
                        alpha_0 = 0,
                        e = 0.25,
                        Malphadot = -1.2,
                        g = 9.81)

In [10]:
# create the numeric model from the symbolic model
nModel = sModel.createNumericInstance(p)

In [11]:
(I_xx+I_yy)*G*1.345

In [12]:
nModel.M

array([[0.55, 0.  , 0.  ],
       [0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  ]])

In [13]:
sModel.M.subs(sModel._s_t,0)

⎡c⋅m_w⋅s_w      ⎤
⎢─────────  0  0⎥
⎢    5          ⎥
⎢               ⎥
⎢    0      0  0⎥
⎢               ⎥
⎣    0      0  0⎦

In [14]:
p.c*p.s_w**4*(p.m_w*p.s_w)/5

In [15]:
vals,vecs = aShape.eig(nModel.K,nModel.M)

In [16]:
omega = np.sqrt(vals,where = ~np.isnan(vals))

In [17]:
omega

array([13.32192658+0.j,  0.        +0.j,  0.        +0.j])

In [18]:
4.47*np.sqrt(E*I_xx/(wingMass*semiSpan**3))